<a href="https://colab.research.google.com/github/d-maniatakos/vaccine-sentiment-classifier/blob/master/vaccine_sentiment_classifier_using_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Vaccine Sentiment Classifier using Recurrent Neural Networks (LSTM/GRU)**


## **Modules Import**

In [1]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, confusion_matrix, ConfusionMatrixDisplay, roc_curve, RocCurveDisplay, roc_auc_score
from sklearn.preprocessing import label_binarize

import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## **Glove Download & Dictionary Creation**


In [2]:
# don't run this cell if file "glove.6B.100d.txt" already exists

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-01-12 08:28:41--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-01-12 08:28:41--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-01-12 08:28:41--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [3]:
glove = {}

# read file line by line while creating glove dictionary
with open('glove.6B.100d.txt') as f:
    lines = f.readlines()
for line in lines:
  vector = []
  for dim in line.split(' ')[1:]:
    vector.append(float(dim))
  glove[line.split(' ')[0]] = np.array(vector)

## **Data Import**


In [4]:
# read datasets and ignore the first (index) column
train_data =  pd.read_csv('vaccine_train_set.csv').iloc[:, 1:]
validation_data = pd.read_csv('vaccine_validation_set.csv').iloc[:, 1:]    # <--- replace with test set path

train_data.head()

,tweet,label
0,Sip N Shop Come thru right now #Marjais #Popul...,0
1,I don't know about you but My family and I wil...,1
2,@MSignorile Immunizations should be mandatory....,2
3,President Obama spoke in favor of vaccination ...,0
4,"""@myfoxla: Arizona monitoring hundreds for mea...",0


## **Pre-processing & Vectorization**


In [37]:
# preprocess tweets texts before vectorization
def preprocess(text):
  text = text.lower()
  unwanted_chars = ['@', '#', '!', '(', ')', '*', ':', ',']

  for char in unwanted_chars:
    text = text.replace(char, '')
  lemmatizer = WordNetLemmatizer()

  lemmatized_text = ''
  for word in text.split():
    lemmatized_text += lemmatizer.lemmatize(word) + ' '
  return lemmatized_text


def padding(original_list, element, length):
  if len(original_list) > length:
    return original_list[:length]
  else:
    new_list = original_list.copy()
    for i in range(len(original_list), length):
      new_list.append(element)
    return new_list


# vectorize a tweet using glove word embedding
def glove_sequence(corpus):
  sequences = []

  for document in corpus:
    sequence = []

    for token in document.split():
      try:
        sequence.append(list(glove[token]))
      except:
        pass

    sequence = padding(sequence, list(np.zeros(100)), 50)
    sequences.append(sequence)
    
  return sequences



train_data['tweet'] = train_data['tweet'].apply(preprocess)
validation_data['tweet'] = validation_data['tweet'].apply(preprocess)

train_corpus = train_data['tweet']
validation_corpus = validation_data['tweet']

x_train = glove_sequence(['review review review', 'good good good', 'review review review', 'good good good'])

In [56]:
class LSTM(nn.Module):
  def __init__(self, x_size, hidden_size, num_of_classes, num_of_layers):
    super(LSTM, self).__init__()
    self.hidden_size = hidden_size

    self.lstm = nn.LSTM(x_size, hidden_size, num_of_layers, bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_of_classes)

  def forward(self, x):
    out, (hidden, cell) = self.lstm(x)
    cat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
    out = self.fc(out)
    return out[:, -1, :]

In [57]:
x_train = torch.Tensor(glove_sequence(train_corpus[:1000]))
y_train = torch.tensor(train_data['label'][:1000].values, dtype=torch.long)

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

x_size = 100
hidden_size = 200
num_of_classes = 3
num_of_layers = 2
learning_rate = 0.05
num_of_epochs = 10
loss_function = nn.CrossEntropyLoss()

model = LSTM(x_size, hidden_size, num_of_classes, num_of_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



for epoch in range(num_of_epochs):
  batch_losses = []
  for index, (x, y) in enumerate(train_dataloader):
    y_pred = model(x)
    loss = loss_function(y_pred, y)
    batch_losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print('Epoch: ' + str(epoch+1) + '/' + str(num_of_epochs) + ' Training Loss: ' + str(round(sum(batch_losses)/len(train_dataloader), 4)))



Epoch: 1/10 Training Loss: 1.8493
Epoch: 2/10 Training Loss: 1.2682
Epoch: 3/10 Training Loss: 1.0757
Epoch: 4/10 Training Loss: 1.0639
Epoch: 5/10 Training Loss: 1.0909
Epoch: 6/10 Training Loss: 1.1848
Epoch: 7/10 Training Loss: 1.1087
Epoch: 8/10 Training Loss: 1.068
Epoch: 9/10 Training Loss: 1.0436
Epoch: 10/10 Training Loss: 1.1234


In [58]:
model(x_train)

tensor([[1.4723, 0.0647, 1.0337],
        [1.7807, 0.2138, 1.1792],
        [1.8389, 0.2287, 1.2056],
        ...,
        [1.8471, 0.2319, 1.2092],
        [1.8419, 0.2372, 1.2051],
        [1.8597, 0.2710, 1.1507]], grad_fn=<SliceBackward0>)

In [59]:
y_train[:10]

tensor([0, 1, 2, 0, 0, 2, 0, 0, 2, 0])